<h1><center></center></h1>
<h1><center>Elevvo Internship</center></h1>
<h1><center>Task 6</center></h1>
<h2><center>Question Answering with Transformers</center></h2>

# **Hands on Task 6**

In this notebook, we experiment with the BERT Base (uncased) model for QA.

**⚙️ Steps**

- Data preprocessing.

- Tokenize using BertTokenizer.

- Fine-tune BertForQuestionAnswering.

- Evaluate performance (compare with DistilBERT).

- Save model in folder: qa_model_bert-base-uncased/.

# **1- Data Collection**

**Setup**

In [1]:
!pip install transformers datasets accelerate evaluate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.2 MB/s eta 0:00:00


**Import required libraries**

In [2]:
import random
import numpy as np
import torch
from datasets import load_dataset

**Set random seeds and device**

In [3]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

**Load the SQuAD v1.1 dataset from Hugging Face**

In [4]:
dataset = load_dataset("squad")
dataset

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

**Preview one sample**

In [5]:
sample = dataset["train"][0]
{k: sample[k] for k in ["context", "question", "answers"]}

{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

**Check average context and answer lengths**

In [6]:
def avg_len(split):
    ctx = np.mean([len(x["context"].split()) for x in split])
    ans = np.mean([len(x["answers"]["text"][0].split()) for x in split])
    return round(ctx, 1), round(ans, 2)

avg_ctx_train, avg_ans_train = avg_len(dataset["train"])
avg_ctx_val,   avg_ans_val   = avg_len(dataset["validation"])
(avg_ctx_train, avg_ans_train, avg_ctx_val, avg_ans_val)

(np.float64(119.8), np.float64(3.16), np.float64(124.0), np.float64(3.02))

**Save smaller subsets for quick tests**

In [7]:
small_train = dataset["train"].select(range(3000))
small_val   = dataset["validation"].select(range(500))
len(small_train), len(small_val)

(3000, 500)

# **2. Preprocessing**

**Import tokenizer**

In [8]:
from transformers import AutoTokenizer

**Choose all model**

In [9]:
MODEL_NAME = "bert-base-uncased"
tokenizer  = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.name_or_path

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

'bert-base-uncased'

**Tokenization setup**

In [10]:
def safe_lengths(tok, desired_max=384, desired_stride=128):
    m = tok.model_max_length if isinstance(tok.model_max_length, int) else 512
    max_len = min(desired_max, m)
    stride  = min(desired_stride, max_len - 32) if max_len > 32 else max(1, max_len // 4)
    return max_len, stride

pad_on_right = tokenizer.padding_side == "right"
max_length, doc_stride = safe_lengths(tokenizer)
max_length, doc_stride, pad_on_right

(384, 128, True)

**training feature builder (per tokenizer)**

In [11]:
def make_prepare_train_features(tok):
    pad_on_right = tok.padding_side == "right"
    max_length, doc_stride = safe_lengths(tok)

    def _fn(examples):
        tokenized = tok(
            examples["question" if pad_on_right else "context"],
            examples["context"  if pad_on_right else "question"],
            truncation=True,
            max_length=max_length,
            stride=doc_stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length"
        )

        sample_map    = tokenized.pop("overflow_to_sample_mapping")
        offset_mapping = tokenized.pop("offset_mapping")

        start_positions, end_positions = [], []

        for i, offsets in enumerate(offset_mapping):
            input_ids = tokenized["input_ids"][i]
            cls_index = input_ids.index(tok.cls_token_id) if tok.cls_token_id in input_ids else 0
            sample_idx = sample_map[i]
            answers = examples["answers"][sample_idx]

            if len(answers["answer_start"]) == 0:
                start_positions.append(cls_index)
                end_positions.append(cls_index)
                continue

            start_char = answers["answer_start"][0]
            end_char   = start_char + len(answers["text"][0])

            sequence_ids = tokenized.sequence_ids(i)

            idx = 0
            ctx_id = 1 if pad_on_right else 0
            while idx < len(sequence_ids) and sequence_ids[idx] != ctx_id:
                idx += 1
            context_start = idx
            while idx < len(sequence_ids) and sequence_ids[idx] == ctx_id:
                idx += 1
            context_end = idx - 1

            if context_start >= len(offsets) or context_end < 0:
                start_positions.append(cls_index)
                end_positions.append(cls_index)
                continue

            if not (offsets[context_start] and offsets[context_start][0] <= start_char and
                    offsets[context_end]   and offsets[context_end][1]   >= end_char):
                start_positions.append(cls_index)
                end_positions.append(cls_index)
            else:
                while context_start < len(offsets):
                    o = offsets[context_start]
                    if o and o[0] <= start_char < o[1]:
                        break
                    context_start += 1

                while context_end >= 0:
                    o = offsets[context_end]
                    if o and o[0] < end_char <= o[1]:
                        break
                    context_end -= 1

                start_positions.append(context_start if context_start >= 0 else cls_index)
                end_positions.append(context_end   if context_end   >= 0 else cls_index)

        tokenized["start_positions"] = start_positions
        tokenized["end_positions"]   = end_positions
        return tokenized

    return _fn

**validation feature builder (per tokenizer)**

In [12]:
def make_prepare_validation_features(tok):
    pad_on_right = tok.padding_side == "right"
    max_length, doc_stride = safe_lengths(tok)

    def _fn(examples):
        tokenized = tok(
            examples["question" if pad_on_right else "context"],
            examples["context"  if pad_on_right else "question"],
            truncation=True,
            max_length=max_length,
            stride=doc_stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length"
        )

        sample_map = tokenized.pop("overflow_to_sample_mapping")
        tokenized["example_id"] = []

        for i in range(len(tokenized["input_ids"])):
            sample_idx = sample_map[i]
            tokenized["example_id"].append(examples["id"][sample_idx])

            sequence_ids = tokenized.sequence_ids(i)
            ctx_id = 1 if pad_on_right else 0
            offsets = tokenized["offset_mapping"][i]
            tokenized["offset_mapping"][i] = [
                o if sequence_ids[k] == ctx_id else None
                for k, o in enumerate(offsets)
            ]

        return tokenized

    return _fn

**Map features for every model**

In [13]:
train_features = dataset["train"].map(
    make_prepare_train_features(tokenizer),
    batched=True,
    remove_columns=dataset["train"].column_names
)

validation_features = dataset["validation"].map(
    make_prepare_validation_features(tokenizer),
    batched=True,
    remove_columns=dataset["validation"].column_names
)

len(train_features), len(validation_features)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

(88524, 10784)

**Quick sanity check**

In [14]:
example_train = train_features[0]
example_val   = validation_features[0]
list(example_train.keys()), list(example_val.keys())

(['input_ids',
  'token_type_ids',
  'attention_mask',
  'start_positions',
  'end_positions'],
 ['input_ids',
  'token_type_ids',
  'attention_mask',
  'offset_mapping',
  'example_id'])

# **3. Model & Training — BERT**


**Import Libraries**

In [15]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer, default_data_collator
import evaluate
import numpy as np
import pandas as pd
import torch

**SQuAD metric and small constants**

In [16]:
metric = evaluate.load("squad")
n_best = 20
max_answer_len = 30

**Post‑processing to convert logits → text spans**

In [17]:
def postprocess_qa_predictions(examples, features, raw_predictions):
    start_logits, end_logits = raw_predictions
    id2idx = {k: i for i, k in enumerate(examples["id"])}
    feats_per_ex = {}
    for i, f in enumerate(features):
        ex_idx = id2idx[f["example_id"]]
        feats_per_ex.setdefault(ex_idx, []).append(i)

    preds = []
    for ex_idx, ex in enumerate(examples):
        ctx = ex["context"]
        f_idxs = feats_per_ex.get(ex_idx, [])
        cand = []
        for fi in f_idxs:
            s_log = start_logits[fi]
            e_log = end_logits[fi]
            offsets = features[fi]["offset_mapping"]
            s_idx = np.argsort(s_log)[-n_best:]
            e_idx = np.argsort(e_log)[-n_best:]
            for s in s_idx:
                for e in e_idx:
                    if s >= len(offsets) or e >= len(offsets):
                        continue
                    if offsets[s] is None or offsets[e] is None:
                        continue
                    if e < s or (e - s + 1) > max_answer_len:
                        continue
                    score = float(s_log[s] + e_log[e])
                    cand.append((score, offsets[s][0], offsets[e][1]))
        if not cand:
            preds.append({"id": ex["id"], "prediction_text": ""})
        else:
            best = max(cand, key=lambda x: x[0])
            preds.append({"id": ex["id"], "prediction_text": ctx[best[1]:best[2]]})
    return preds

**Metric callback factory per model**

In [34]:
def compute_metrics_single(eval_pred):
    # Access the predictions attribute from the EvalPrediction object
    raw = eval_pred.predictions if hasattr(eval_pred, 'predictions') else eval_pred
    preds = postprocess_qa_predictions(
        dataset["validation"],      # original examples
        validation_features,        # tokenized features
        raw
    )
    refs = [{"id": ex["id"], "answers": ex["answers"]} for ex in dataset["validation"]]
    return metric.compute(predictions=preds, references=refs)

**TrainingArguments helper that works on old and new transformers**

In [19]:
def make_args(output_stem, lr=3e-5, bs=16, epochs=2, fp16=torch.cuda.is_available()):
    out = f"./qa_runs_{output_stem.replace('/', '_')}"
    try:
        return TrainingArguments(
            output_dir=out,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=lr,
            per_device_train_batch_size=bs,
            per_device_eval_batch_size=bs,
            num_train_epochs=epochs,
            weight_decay=0.01,
            fp16=fp16,
            logging_steps=100,
            report_to="none",
            load_best_model_at_end=True,
            metric_for_best_model="f1",
            greater_is_better=True
        )
    except TypeError:
        # older transformers fallback
        return TrainingArguments(
            output_dir=out,
            do_train=True,
            do_eval=True,
            learning_rate=lr,
            per_device_train_batch_size=bs,
            per_device_eval_batch_size=bs,
            num_train_epochs=epochs,
            weight_decay=0.01,
            fp16=fp16,
            logging_steps=100,
            report_to="none",
            save_total_limit=2,
            save_steps=1000,
            eval_steps=1000
        )

**Build trainer**

In [20]:
def build_trainer_single(model, args):
    try:
        return Trainer(
            model=model,
            args=args,
            train_dataset=train_features,
            eval_dataset=validation_features,
            processing_class=tokenizer,
            data_collator=default_data_collator,
            compute_metrics=compute_metrics_single
        )
    except TypeError:
        return Trainer(
            model=model,
            args=args,
            train_dataset=train_features,
            eval_dataset=validation_features,
            tokenizer=tokenizer,
            data_collator=default_data_collator,
            compute_metrics=compute_metrics_single
        )

In [21]:
name = "bert-base-uncased"
print(f"=== Training {name} ===")

model = AutoModelForQuestionAnswering.from_pretrained(name)
args  = make_args(name)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_features,
    eval_dataset=validation_features,
    tokenizer=tokenizer,                   # or processing_class=tokenizer on newer versions
    data_collator=default_data_collator,
    compute_metrics=compute_metrics_single
)

trainer.train()
metrics = trainer.evaluate()
metrics

=== Training bert-base-uncased ===


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1032791606.py:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
100,3.717100
200,2.298700
300,1.997200
400,1.720600
500,1.645300
600,1.529400
700,1.475800
800,1.505200
900,1.419500
1000,1.343600


{'eval_runtime': 58.5588,
 'eval_samples_per_second': 184.157,
 'eval_steps_per_second': 11.51,
 'epoch': 2.0}

**Save fine‑tuned model + tokenizer**

In [22]:
save_dir = f"./qa_model_{name.replace('/', '_')}"
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)

pd.DataFrame([{"model": name, **{k: float(v) for k, v in metrics.items()}}])

,model,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
0,bert-base-uncased,58.5588,184.157,11.51,2.0


In [23]:
df_results = pd.DataFrame([{"model": name, **{k: float(v) for k, v in metrics.items()}}])
df_results

,model,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
0,bert-base-uncased,58.5588,184.157,11.51,2.0


# **4. Inference & Quick Sanity Check**

**Load the fine‑tuned checkpoint for this model**

In [24]:
from transformers import pipeline

MODEL_NAME = "bert-base-uncased"
CKPT_DIR   = f"./qa_model_{MODEL_NAME.replace('/', '_')}"
tokenizer  = AutoTokenizer.from_pretrained(CKPT_DIR, use_fast=True)
model      = AutoModelForQuestionAnswering.from_pretrained(CKPT_DIR)

**Build the QA pipeline**

In [25]:
qa_pipeline = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

Device set to use cuda:0


**Example context and question**

In [26]:
context = """
The Nile River is the longest river in the world, flowing northward through
eastern Africa into the Mediterranean Sea. It has historically been of great
importance to Egyptian civilization.
"""
question = "Which river is the longest in the world?"

result = qa_pipeline(question=question, context=context)
print("Question:", question)
print("Answer:", result["answer"])
print("Score:", round(result["score"], 4))

Question: Which river is the longest in the world?
Answer: Nile River
Score: 0.4285


# **5. Export (checkpoint)**

**Set paths for this model**

In [27]:
import os, json, pathlib
MODEL_NAME = "bert-base-uncased"
CKPT_DIR   = f"./qa_model_{MODEL_NAME.replace('/', '_')}"
pathlib.Path(CKPT_DIR).mkdir(parents=True, exist_ok=True)
CKPT_DIR

'./qa_model_bert-base-uncased'

**Save final checkpoint (model + tokenizer + a mini card)**

In [28]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# Re-save to be sure the latest trained weights are on disk
_ = AutoModelForQuestionAnswering.from_pretrained(CKPT_DIR).save_pretrained(CKPT_DIR)
_ = AutoTokenizer.from_pretrained(CKPT_DIR).save_pretrained(CKPT_DIR)

with open(os.path.join(CKPT_DIR, "model_card.txt"), "w") as f:
    f.write(f"Model: {MODEL_NAME}\nNotes: Fine-tuned on SQuAD v1.1 for extractive QA\n")

**Write metrics JSON (EM/F1)**

In [37]:
metrics_out = {
    "model": MODEL_NAME,
    "metrics": {k: float(v) for k, v in all_metrics.items()}
}
with open(f"./metrics_{MODEL_NAME.replace('/', '_')}.json", "w") as f:
    json.dump(metrics_out, f, indent=2)

metrics_out

{'model': 'bert-base-uncased',
 'metrics': {'eval_model_preparation_time': 0.0118,
  'eval_runtime': 58.089,
  'eval_samples_per_second': 185.646,
  'eval_steps_per_second': 11.603,
  'exact_match': 81.01229895931883,
  'f1': 88.35681807945764}}